# Q1

In [1]:
!pip install fastembed

In [2]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')
embedding_model.model_name

'jinaai/jina-embeddings-v2-small-en'

In [4]:
query = 'I just discovered the course. Can I join now?'
query

'I just discovered the course. Can I join now?'

In [5]:
query_embedded = list(embedding_model.embed([query]))
query_embedded[0].shape

(512,)

In [6]:
print(query_embedded[0].min())

-0.11726373885183883


In [7]:
import numpy as np
np.linalg.norm(query_embedded[0])

np.float64(1.0)

In [8]:
print(query_embedded[0].dot(query_embedded[0]))

1.0000000000000002


# Q2

In [9]:
doc = 'Can I still join the course after the start date?'
doc

'Can I still join the course after the start date?'

In [10]:
doc_embedded = list(embedding_model.embed([doc]))
print(query_embedded[0].dot(doc_embedded[0]))

0.9008528895674548


# Q3

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
documents_embedded = list(embedding_model.embed([doc['text'] for doc in documents]))
similarities = np.stack(documents_embedded, axis=0).dot(query_embedded[0])
print(similarities)

[0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]


In [13]:
[float(x.dot(query_embedded[0])) for x in documents_embedded]

[0.7629684696540238,
 0.8182378150042889,
 0.8085397398734189,
 0.7133079015686243,
 0.7304499234333598]

In [14]:
(lambda x : (x, documents[x]['text']))(np.argmax(similarities))

(np.int64(1),
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.')

# Q4

In [15]:
for doc in documents:
    doc['full_text'] = doc['question'] + ' ' + doc['text']

documents_embedded2 = list(embedding_model.embed([doc['full_text'] for doc in documents]))
similarities2 = np.stack(documents_embedded2, axis=0).dot(query_embedded[0])
print(similarities2)

[0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]


In [16]:
(lambda x : (x, documents[x]['text']))(np.argmax(similarities2))

(np.int64(0),
 "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.")

# Q5

In [17]:
import pandas as pd
df_supported_models = pd.DataFrame.from_records(TextEmbedding.list_supported_models())
df_supported_models.sort_values(by='dim')[['dim', 'model']].head()

,dim,model
3,384,BAAI/bge-small-en
4,384,BAAI/bge-small-en-v1.5
7,384,snowflake/snowflake-arctic-embed-xs
8,384,snowflake/snowflake-arctic-embed-s
14,384,sentence-transformers/all-MiniLM-L6-v2


In [18]:
smallest_dim = df_supported_models['dim'].min()
print(smallest_dim)

384


In [19]:
model_handle = 'BAAI/bge-small-en'
print(model_handle)

BAAI/bge-small-en


# Q6

In [20]:
import requests
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
documents[:3]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [22]:
from qdrant_client import QdrantClient
client = QdrantClient('http://localhost:6333')
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='vector+rag'), CollectionDescription(name='sparse'), CollectionDescription(name='homework'), CollectionDescription(name='sparse+dense'), CollectionDescription(name='zoomcamp-rag')])

In [23]:
from qdrant_client import models
client.delete_collection(collection_name='homework')
client.create_collection(
	collection_name='homework',
	vectors_config=models.VectorParams(
		size=smallest_dim,
		distance=models.Distance.COSINE
	)
)

True

In [24]:
points = []

for id, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=id,
        vector=vector,
        payload=doc
    )
    points.append(point)

len(points)

375

In [25]:
points[:3]

[PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I 

In [26]:
from datetime import datetime
print(datetime.now())
client.upsert(
	collection_name='homework',
	points=points
)
print(datetime.now())

2025-06-27 05:25:11.008557
2025-06-27 05:26:24.336143


In [27]:
query_points = client.query_points(
    collection_name='homework',
    query=models.Document(
        text=query,
        model=model_handle
    ),
    limit=1,
    with_payload=True
)

In [28]:
query_points.points[0].payload

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp'}

In [29]:
query_points.points[0].score

0.8703172